In [1]:
# data stuff:
import pandas as pd
import numpy as np

# plotting stuff:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image

# stats stuff:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# modeling stuff:
from sklearn.metrics.pairwise import cosine_similarity

# silence!
import warnings
warnings.filterwarnings("ignore")

---
## 📊 Collaborative Filtering:
Below we cover:

* Item-based Collaborative Filtering
* User-based Collaborative Filtering





**📄 Dataset:** UCI Online Retail Data Set (https://archive.ics.uci.edu/ml/datasets/online+retail)

📚 See chapter 6 of _"Data Science for Marketing"_ by Hwang, page 198.

---

### Import Data and Preprocess:

In [2]:
data = pd.read_csv('data/Online Retail.csv')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France


&nbsp;
#### Remove Missing Customer Ids:

In [3]:
# check for missing Values:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [4]:
# drop customer Ids that are missing and remove negative quantities:
data = data.dropna(subset=['CustomerID'])
data = data.loc[data['Quantity'] > 0]
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France


---
### Build Customer-Item Matrix:
Below we use the customers as rows and items as columns.

&nbsp;
#### Pivot Data:

In [5]:
# pivot data to obtain matrix of product counts:
cust_item_df = data.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc=sum)
cust_item_df

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


&nbsp;
#### Create Unary Matrix:
Flatten counts into purchase indicator - 1 if bought, else 0.

In [6]:
cust_item_df = cust_item_df.applymap(lambda x: 1 if x > 0 else 0)
cust_item_df

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18281.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18282.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---
### User-based Collaborative Filtering:
First, compute cosine similarity between users.

In [7]:
user_user_sim_matrix = pd.DataFrame(cosine_similarity(cust_item_df), columns=cust_item_df.index, index=cust_item_df.index)
user_user_sim_matrix

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044866,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.062318,0.000000,0.000000,0.000000,0.000000,0.0,0.041523,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.105409,1.000000,0.119523,0.000000,0.000000,0.000000
18281.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.049629,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.119523,1.000000,0.000000,0.046613,0.000000
18282.0,0.0,0.113776,0.000000,0.067574,0.000000,0.037582,0.0,0.000000,0.160128,0.079305,...,0.0,0.174078,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.017800,0.000000


&nbsp;
#### Find similar users:
Let's take user 12350 as our test subject:

In [8]:
# call user's index and sort:
user_user_sim_matrix.loc[12350.0].sort_values(ascending=False)

CustomerID
12350.0    1.000000
17935.0    0.183340
12414.0    0.181902
12652.0    0.175035
16692.0    0.171499
             ...   
14885.0    0.000000
14886.0    0.000000
14887.0    0.000000
14888.0    0.000000
18287.0    0.000000
Name: 12350.0, Length: 4339, dtype: float64

&nbsp;
#### Find items to recommend:
Find items that user 17935.0 has not yet bought by comparing it against user 12350:

In [12]:
user_a_items = cust_item_df.loc[12350.0].to_frame().reset_index()
user_b_items = cust_item_df.loc[17935.0].to_frame().reset_index()
user_b_items

,StockCode,17935.0
0,10002,0
1,10080,0
2,10120,0
3,10123C,0
4,10124A,0
...,...,...
3660,C2,0
3661,DOT,0
3662,M,0
3663,PADS,0


In [10]:
# we do a left merge to keep all the items in user-a that have not been purchased by user-b (if they exist):
rec_items = pd.merge(left=a, right=b, how='left', left_on='StockCode', right_on='StockCode')

In [23]:
# now, find all items in the rec matrix that user-a has bought and user-b hasn't
recommend_to_b = rec_items[(rec_items[17935.0]==0) & (rec_items[12350.0]==1)]
recommend_to_b_items = list(recommend_to_b['StockCode'])

['20615',
 '20652',
 '21171',
 '21832',
 '21864',
 '21908',
 '21915',
 '22348',
 '22412',
 '22620',
 '79066K',
 '79191C',
 '84086C']

In [35]:
recommend_to_b_items = data[data['StockCode'].isin(recommend_to_b_items)][['Description', 'StockCode']].drop_duplicates()
recommend_to_b_items.set_index('StockCode')

,Description
StockCode,
21832,CHOCOLATE CALCULATOR
21915,RED HARMONICA IN BOX
22620,4 TRADITIONAL SPINNING TOPS
79066K,RETRO MOD TRAY
21864,UNION JACK FLAG PASSPORT COVER
79191C,RETRO PLASTIC ELEPHANT TRAY
21908,CHOCOLATE THIS WAY METAL SIGN
20615,BLUE POLKADOT PASSPORT COVER
20652,BLUE POLKADOT LUGGAGE TAG


&nbsp;

---
### Item-Item Collaborative Filtering:
Since we already computed the cosine similarity between items, we'll make use of the item-user matrix.

In [43]:
# transpose
item_item_matrix = cust_item_df.T

# find similarity:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(item_item_matrix), index=item_item_matrix.index, columns=item_item_matrix.index)
item_item_sim_matrix

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,...,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.066915,0.0,0.078217
10080,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016182,0.0,0.000000
10120,0.094868,0.000000,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.0,0.010993
10123C,0.091287,0.000000,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.000000,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,0.029361,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.036955,0.019360,0.055989,...,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.026196,0.0,0.020413
DOT,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.104257,0.150756,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000
M,0.066915,0.016182,0.070535,0.000000,0.0,0.000000,0.070535,0.070185,0.066184,0.106335,...,0.049875,0.0,0.040723,0.0,0.089220,0.026196,0.0,1.000000,0.0,0.077539


&nbsp;
#### Find similar items:
Let's take item 23166 as our test subject and find the top 10 most similar items:

In [48]:
top_ten = item_item_sim_matrix.loc['23166'].sort_values(ascending=False)[:10]
top_ten

StockCode
23166    1.000000
23165    0.671897
23167    0.659551
22993    0.274724
23307    0.262526
22722    0.249053
22720    0.249002
22666    0.247110
23243    0.246397
22961    0.242447
Name: 23166, dtype: float64

&nbsp;
#### Return top-10 descriptions:

In [51]:
data[data['StockCode'].isin(top_ten.index)][['StockCode', 'Description']].drop_duplicates()

,StockCode,Description
105,22961,JAM MAKING SET PRINTED
1296,22666,RECIPE BOX PANTRY YELLOW DESIGN
28714,22722,SET OF 6 SPICE TINS PANTRY DESIGN
28715,22720,SET OF 3 CAKE TINS PANTRY DESIGN
61619,23166,MEDIUM CERAMIC TOP STORAGE JAR
92216,22993,SET OF 4 PANTRY JELLY MOULDS
185919,23167,SMALL CERAMIC TOP STORAGE JAR
186769,23165,LARGE CERAMIC TOP STORAGE JAR
212379,23307,SET OF 60 PANTRY DESIGN CAKE CASES
217655,23243,SET OF TEA COFFEE SUGAR TINS PANTRY


As we can see above, the people that buy "Jam Making Set" tend to buy recipe boxes, cake tins, storage jars, etc. So it would be a great idea to recommend these to the people looking for Jam Making Sets.